In [ ]:
# %%
path = '/home/wjunneng/Python/ForecastScore/'

submission_s1 = '/data/test_s1/submission_s1.csv'
submission_s1_sample_mean_median = '/data/test_s1/submission_s1_sample_mean_median.csv'
submission_s1_sample_svm = '/data/test_s1/submission_s1_sample_svm.csv'
submission_s1_sample_xgb = '/data/test_s1/submission_s1_sample_xgb.csv'
submission_s1_sample_lgb = '/data/test_s1/submission_s1_sample_lgb.csv'

submission_s1_sample_mean_median1 = '/data/test_s1/submission_s1_sample_mean_median1.csv'

course = '/data/train_s1/course.csv'
student = '/data/train_s1/student.csv'
exam_score = '/data/train_s1/exam_score.csv'
all_knowledge = '/data/train_s1/all_knowledge.csv'
course1_exams = '/data/train_s1/course1_exams.csv'
course2_exams = '/data/train_s1/course2_exams.csv'
course3_exams = '/data/train_s1/course3_exams.csv'
course4_exams = '/data/train_s1/course3_exams.csv'
course5_exams = '/data/train_s1/course5_exams.csv'
course6_exams = '/data/train_s1/course6_exams.csv'
course7_exams = '/data/train_s1/course7_exams.csv'
course8_exams = '/data/train_s1/course8_exams.csv'

submission_s1_sample = '/data/submission_s1_sample.csv'
# %%
import numpy as np
import pandas as pd


def get_train_s1(file_name, tag):
    """
    返回train_s1文件夹下的文件
    :param file_name: 文件名称
    :param tag: 返回的数据类型
    :return:
    """
    if tag == 'np':
        return np.loadtxt(path + file_name, delimiter=',', dtype=np.str)
    elif tag == 'pd':
        return pd.read_csv(path + file_name)
    else:
        return "请检查文件名/需要返回的数据类型"


def get_test_s1(file_name, tag):
    """
    返回test_s1文件夹下的文件
    :param file_name: 文件名称
    :param tag: 返回的数据类型
    :return:
    """
    if tag == 'np':
        return np.loadtxt(path + file_name, delimiter=',', dtype=np.str)
    elif tag == 'pd':
        return pd.read_csv(path + file_name)
    else:
        return "请检查文件名/需要返回的数据类型"


def get_sample(file_name, tag):
    """
    返回范例文件夹下的文件
    :param file_name: 文件名称
    :param tag: 返回的数据类型
    :return:
    """
    if tag == 'np':
        return np.loadtxt(path + file_name, delimiter=',', dtype=np.str)
    elif tag == 'pd':
        return pd.read_csv(path + file_name)
    else:
        return "请检查文件名/需要返回的数据类型"


# %%
exam_score_data = get_train_s1(exam_score, 'pd')

reuslt = []
for i in ['course1', 'course2', 'course3', 'course4', 'course5', 'course6', 'course7', 'course8']:
    exam_score_course = exam_score_data[exam_score_data.course == i]
    for exam_score_course_student in exam_score_course.groupby('student_id'):
        print('--' + str(exam_score_course_student[1].shape) + '--')
        tmp = exam_score_course_student[1].score
        if tmp.values.min() == 0:
            print(exam_score_course_student[1].shape)

print(exam_score_data.describe())
print(exam_score_data.columns)


# %%
def reduce_mem_usage(df, verbose=True):
    """
    减少内存消耗
    :param df:
    :param verbose:
    :return:
    """
    numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
    start_mem = df.memory_usage().sum() / 1024 ** 2
    for col in df.columns:
        col_type = df[col].dtypes
        if col_type in numerics:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)
    end_mem = df.memory_usage().sum() / 1024 ** 2
    if verbose: print('Mem. usage decreased to {:5.2f} Mb ({:.1f}% reduction)'.format(end_mem, 100 * (
            start_mem - end_mem) / start_mem))
    return df


# %%
# 运行速度慢  暂时丢弃
import time

reuslt = []

start = time.clock()
for i in ['1', '2', '3', '4', '5', '6', '7', '8']:
    print(time.clock() - start)

    # submission_path = path + '/data/cache/submission_s1_course' + i + '.h5'
    # test_X = reduce_mem_usage(pd.read_hdf(path_or_buf=submission_path, mode='r', key='course'+i))
    # del test_X['pred']

    exam_score_path = path + '/data/cache/exam_score_course' + i + '.h5'
    train_X = reduce_mem_usage(pd.read_hdf(path_or_buf=exam_score_path, mode='r', key='course' + i))

    import lightgbm as lgb
    from sklearn.model_selection import KFold, GridSearchCV

    # LGBMRegressor
    model = lgb.LGBMRegressor(boosting_type='dart', random_state=50,
                              objective='regression', subsample=0.6143)

    folds = KFold(n_splits=5, shuffle=True, random_state=1)

    params_spaces = {
        'learning_rate': (0.01, 0.5),
        'num_leaves': (10, 100),
        'max_depth': (30, 100),
        'min_child_samples': (0, 50),
        'max_bin': (100, 1000),
        'subsample_freq': (0, 10),
        'min_child_weight': (0, 10),
        'reg_lambda': (1e-9, 100),
        'reg_alpha': (1e-9, 1.0),
        'scale_pos_weight': (1e-6, 100),
        'n_estimators': (100, 400),
    }

    gs_cv_tuner = GridSearchCV(
        estimator=model,
        param_grid=params_spaces,
        scoring='neg_mean_squared_error',
        cv=folds,
        verbose=0,
        refit=True,
        return_train_score=True,
        n_jobs=8)

    # 获取exam_id
    exam_id = list(set(train_X.exam_id))
    # 对exam_id进行排序
    exam_id.sort()
    # 获取训练集
    train_X = train_X[train_X.exam_id.isin(exam_id[:int(len(exam_id) - 2)])]
    # 获取测试集
    test_X = train_X[train_X.exam_id.isin(exam_id[int(len(exam_id) - 2):])]

    train_y = train_X['score']
    test_y = test_X['score']

    del train_X['score']
    del test_X['score']

    result = gs_cv_tuner.fit(train_X, train_y)

    print("val. score: %s" % result.best_score_)
    print("test score: %s" % result.score(test_X, test_y))
# %%
#######################################  使用验证集验证模型xgb  #####################################################
import pickle
import time
from xgboost import XGBRegressor
from sklearn.model_selection import RandomizedSearchCV

models = []

start = time.clock()
for i in ['1', '2', '3', '4', '5', '6', '7', '8']:
    print('耗时：%s \n' % str(time.clock() - start))

    exam_score_path = path + '/data/cache/exam_score_course' + i + '.h5'
    train_X = reduce_mem_usage(pd.read_hdf(path_or_buf=exam_score_path, mode='r', key='course' + i))

    #######################################  训练集-2  ############################################################
    # 获取exam_id
    exam_id = list(set(train_X.exam_id))
    # 对exam_id进行排序
    exam_id.sort()
    # 获取训练集
    train_X = train_X[train_X.exam_id.isin(exam_id[:int(len(exam_id) - 2)])]
    #######################################  训练集-2  ############################################################

    train_y = train_X['score']
    del train_X['score']

    model = XGBRegressor(boosting_type='dart')
    search_params = {
        'max_depth': list(range(1, 10, 1)),
        'n_estimators': list(range(10, 1000, 10)),
        'gamma': list(np.arange(0., 1, .05)),
        'min_child_weight': list(np.arange(0.1, 1, .1)),
        'colsample_bytree': list(np.arange(0.1, 1, .05)),
        'learning_rate': list(np.arange(0.01, 0.1, 0.01))
    }
    search = RandomizedSearchCV(model, search_params, n_iter=50, n_jobs=8)

    search.fit(train_X, train_y)
    models.append(search.best_estimator_)
    print(search.best_estimator_)

start = time.clock()

for index, model in enumerate(models):
    print('耗时：%s \n' % str(time.clock() - start))

    index += 1
    exam_score_path = path + '/data/cache/exam_score_course' + str(index) + '.h5'
    train_X = reduce_mem_usage(pd.read_hdf(path_or_buf=exam_score_path, mode='r', key='course' + str(index)))

    #######################################  训练集-2  ############################################################
    # 获取exam_id
    exam_id = list(set(train_X.exam_id))
    # 对exam_id进行排序
    exam_id.sort()
    # 获取训练集
    train_X = train_X[train_X.exam_id.isin(exam_id[:int(len(exam_id) - 2)])]
    #######################################  训练集-2  ############################################################

    train_y = train_X['score']
    del train_X['score']

    model.fit(train_X, train_y)

    with open(path + '/model/xgb_model_' + str(index) + '.pkl', 'wb') as file:
        pickle.dump(model, file)

    print('成功保存xgb_model_' + str(index) + '.pkl!')

start = time.clock()

for index, model_file in enumerate(['xgb_model_1.pkl', 'xgb_model_2.pkl', 'xgb_model_3.pkl',
                                    'xgb_model_4.pkl', 'xgb_model_5.pkl', 'xgb_model_6.pkl',
                                    'xgb_model_7.pkl', 'xgb_model_8.pkl']):
    print('耗时：%s \n' % str(time.clock() - start))

    index += 1
    exam_score_path = path + '/data/cache/exam_score_course' + str(index) + '.h5'
    train_X = reduce_mem_usage(pd.read_hdf(path_or_buf=exam_score_path, mode='r', key='course' + str(index)))

    #######################################  2  ############################################################
    # 获取exam_id
    exam_id = list(set(train_X.exam_id))
    # 对exam_id进行排序
    exam_id.sort()
    # 获取训练集
    test_X = train_X[train_X.exam_id.isin(exam_id[int(len(exam_id) - 2):])]
    #######################################  2  ############################################################

    test_y = test_X['score']
    del test_X['score']

    with open(path + '/model/' + model_file, 'rb') as file:
        model = pickle.load(file)

        predictions = model.predict(test_X)
        test_y = np.array(test_y)
        predictions = np.array(predictions)
        rmse = np.sqrt(((test_y - predictions) ** 2).mean())

        print('rmse：' + str(rmse))

# %%
#######################################  使用验证集验证模型lgb  #####################################################
import pickle
import time
from lightgbm import LGBMRegressor
from sklearn.model_selection import RandomizedSearchCV

models = []

start = time.clock()
for i in ['1', '2', '3', '4', '5', '6', '7', '8']:
    print('耗时：%s \n' % str(time.clock() - start))

    exam_score_path = path + '/data/cache/exam_score_course' + i + '.h5'
    train_X = reduce_mem_usage(pd.read_hdf(path_or_buf=exam_score_path, mode='r', key='course' + i))

    #######################################  训练集-2  ############################################################
    # 获取exam_id
    exam_id = list(set(train_X.exam_id))
    # 对exam_id进行排序
    exam_id.sort()
    # 获取训练集
    train_X = train_X[train_X.exam_id.isin(exam_id[:int(len(exam_id) - 2)])]
    #######################################  训练集-2  ############################################################

    train_y = train_X['score']
    del train_X['score']

    model = LGBMRegressor(boosting_type='dart', objective='regression', n_estimators=1000, max_depth=10,
                          learning_rate=0.1)
    search_params = {
        'max_depth': list(range(1, 10, 1)),
        'n_estimators': list(range(10, 1000, 10)),
        'gamma': list(np.arange(0., 1, .05)),
        'min_child_weight': list(np.arange(0.1, 1, .1)),
        'colsample_bytree': list(np.arange(0.1, 1, .05)),
        'learning_rate': list(np.arange(0.01, 0.1, 0.01))
    }
    search = RandomizedSearchCV(model, search_params, n_iter=100, n_jobs=8)

    search.fit(train_X, train_y)
    models.append(search.best_estimator_)
    print(search.best_estimator_)

start = time.clock()

for index, model in enumerate(models):
    print('耗时：%s \n' % str(time.clock() - start))

    index += 1
    exam_score_path = path + '/data/cache/exam_score_course' + str(index) + '.h5'
    train_X = reduce_mem_usage(pd.read_hdf(path_or_buf=exam_score_path, mode='r', key='course' + str(index)))

    #######################################  训练集-2  ############################################################
    # 获取exam_id
    exam_id = list(set(train_X.exam_id))
    # 对exam_id进行排序
    exam_id.sort()
    # 获取训练集
    train_X = train_X[train_X.exam_id.isin(exam_id[:int(len(exam_id) - 2)])]
    #######################################  训练集-2  ############################################################

    train_y = train_X['score']
    del train_X['score']

    model.fit(train_X, train_y)

    with open(path + '/model/lgb_model_' + str(index) + '.pkl', 'wb') as file:
        pickle.dump(model, file)

    print('成功保存lgb_model_' + str(index) + '.pkl!')

start = time.clock()

for index, model_file in enumerate(['lgb_model_1.pkl', 'lgb_model_2.pkl', 'lgb_model_3.pkl',
                                    'lgb_model_4.pkl', 'lgb_model_5.pkl', 'lgb_model_6.pkl',
                                    'lgb_model_7.pkl', 'lgb_model_8.pkl']):
    print('耗时：%s \n' % str(time.clock() - start))

    index += 1
    exam_score_path = path + '/data/cache/exam_score_course' + str(index) + '.h5'
    train_X = reduce_mem_usage(pd.read_hdf(path_or_buf=exam_score_path, mode='r', key='course' + str(index)))

    #######################################  2  ############################################################
    # 获取exam_id
    exam_id = list(set(train_X.exam_id))
    # 对exam_id进行排序
    exam_id.sort()
    # 获取训练集
    test_X = train_X[train_X.exam_id.isin(exam_id[int(len(exam_id) - 2):])]
    #######################################  2  ############################################################

    test_y = test_X['score']
    del test_X['score']

    with open(path + '/model/' + model_file, 'rb') as file:
        model = pickle.load(file)

        predictions = model.predict(test_X)
        test_y = np.array(test_y)
        predictions = np.array(predictions)
        rmse = np.sqrt(((test_y - predictions) ** 2).mean())

        print('rmse：' + str(rmse))
# %%
#######################################  使用训练集训练模型xgb  #####################################################
import pickle
import time
from xgboost import XGBRegressor
from sklearn.model_selection import RandomizedSearchCV

models = []

start = time.clock()
for i in ['1', '2', '3', '4', '5', '6', '7', '8']:
    print('耗时：%s \n' % str(time.clock() - start))

    exam_score_path = path + '/data/cache/exam_score_course' + i + '.h5'
    train_X = reduce_mem_usage(pd.read_hdf(path_or_buf=exam_score_path, mode='r', key='course' + i))

    train_y = train_X['score']
    del train_X['score']

    model = XGBRegressor(boosting_type='dart')
    search_params = {
        'max_depth': list(range(1, 10, 1)),
        'n_estimators': list(range(10, 1000, 10)),
        'gamma': list(np.arange(0., 1, .05)),
        'min_child_weight': list(np.arange(0.1, 1, .1)),
        'colsample_bytree': list(np.arange(0.1, 1, .05)),
        'learning_rate': list(np.arange(0.01, 0.1, 0.01))
    }
    search = RandomizedSearchCV(model, search_params, n_iter=50, n_jobs=8)

    search.fit(train_X, train_y)
    models.append(search.best_estimator_)
    print(search.best_estimator_)

start = time.clock()

for index, model in enumerate(models):
    print('耗时：%s \n' % str(time.clock() - start))

    index += 1
    exam_score_path = path + '/data/cache/exam_score_course' + str(index) + '.h5'
    train_X = reduce_mem_usage(pd.read_hdf(path_or_buf=exam_score_path, mode='r', key='course' + str(index)))
    train_y = train_X['score']
    del train_X['score']

    model.fit(train_X, train_y)

    with open(path + '/model/xgb_model_' + str(index) + '.pkl', 'wb') as file:
        pickle.dump(model, file)

    print('成功保存xgb_model_' + str(index) + '.pkl!')
# %%

